# Data preprocessing
This notebook preprocesses data downloaded from the Argo data platform to experimental data. The preprocessing precedure follows the following steps: 
- Select relevant columns and drop NAN rows
- Add global QC labels to each row
- Rename columns
- Normalize data (z-score standardization)

See Argo QC flag scale @ ./argo_qc_flag_scale.png


Input: 
- `../data/original_data/Atlantic_2019_03`

Output: 
- `../data/preprocessed_data/Atlantic_2019_03`

In [ ]:
import pandas as pd
import numpy as np
import os
import csv
from datetime import datetime 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
import matplotlib.ticker as mticker

In [ ]:
ORIGINAL_DATA_DIR = '../data/original_data/Atlantic_2019_03'
PREPROCESSED_DATA_DIR = '../data/preprocessed_data/Atlantic_2019_03'
NORMALIZED_DATA_DIR = '../data/preprocessed_data/Atlantic_2019_03/normalized'

os.makedirs(PREPROCESSED_DATA_DIR, exist_ok=True)
os.makedirs(NORMALIZED_DATA_DIR, exist_ok=True)

In [ ]:
# Compute data error rate
def comp_error_ratio(dataset):
    instance = dataset[(dataset['Label']==1)]
    rate=len(instance)/len(dataset)*100
    return round(rate,2) 

In [ ]:
# Add global QC label to each record. 
# If all QC are `1`, assign the label `0`(normal). Otherwise assign the label `1`(abnormal). 
def add_global_qc_label(df):
    values = df[['DATE_QC', 'POSITION_QC', 'PRES_QC', 'PSAL_QC', 'TEMP_QC']]
    is_all_normal = values.isin({1}).all(axis=1)
    label_values = is_all_normal.map(lambda x: 0 if x else 1)
    df['Label'] = label_values 

    print(f'Error rate: {comp_error_ratio(df)}%')
    return df

In [ ]:
# Change the column names
column_map = {
              # 'PLATFORM_CODE': 'Platform_code', 
              'ID': 'ID', 
              'DATE (YYYY-MM-DDTHH:MI:SSZ)': 'Datetime',
              'LATITUDE (degree_north)': 'Latitude', 
              'LONGITUDE (degree_east)': 'Longitude', 
              'PRES (decibar)': 'Pressure', 
              'PSAL (psu)': 'Salinity', 
              'TEMP (degree_Celsius)': 'Temperature', 
              'Label': 'Label', 
              # 'PRES_ADJUSTED (decibar)': 'pressure_adjusted', 
              # '': '', 
             }
def change_column_names(df): 
    df.rename(columns=column_map, inplace=True)
    data = df[list(column_map.values())]
    return data

In [ ]:
# Normalize data
def standardize_columns(df, columns):
    df_standardized = df.copy()
    for column in columns:
        col_mean = df[column].mean()
        col_std = df[column].std()
        df_standardized[column] = (df[column] - col_mean) / col_std
    return df_standardized


In [ ]:
# Normalize date
def normalize_date_column(df, date_column, reference_date=None):
    df_normalized = df.copy()
    df_normalized[date_column] = pd.to_datetime(df_normalized[date_column])
    
    if reference_date is None:
        reference_date = df_normalized[date_column].min()

    df_normalized['DaysSinceReference'] = (df_normalized[date_column] - reference_date).dt.days
    
    col_mean = df_normalized['DaysSinceReference'].mean()
    col_std = df_normalized['DaysSinceReference'].std()
    
    df_normalized.insert(2, 'Normalized_date', (df_normalized['DaysSinceReference'] - col_mean) / col_std)
    # df_normalized['Normalized_date'] = (df_normalized['DaysSinceReference'] - col_mean) / col_std
    
    return df_normalized.drop(columns=['DaysSinceReference'])

# Sample DataFrame with a date column
data = {'Date': ['2022-01-01', '2022-02-01', '2022-03-01', '2022-04-01', '2022-05-01'],
        'Value': [10, 20, 30, 40, 50]}

df = pd.DataFrame(data)

# Normalize the 'Date' column using a reference date of '2010-01-01'
normalized_df = normalize_date_column(df, date_column='Date', reference_date=pd.Timestamp('2010-03-20'))

print("Original DataFrame:")
print(df)

print("\nNormalized DataFrame:")
print(normalized_df)


### Preprocess the data

In [ ]:
input_dir = ORIGINAL_DATA_DIR
output_dir = PREPROCESSED_DATA_DIR
normalized_dir = NORMALIZED_DATA_DIR
file_paths = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith('.csv')]

print(f'Input dir: {input_dir}')
print(f'Output dir: {output_dir}')
for filename in os.listdir(input_dir):
    if filename.endswith(".csv"):
        input_path = os.path.join(input_dir, filename)
        output_path = os.path.join(output_dir, filename)
        normalized_path = os.path.join(normalized_dir, filename)

        df = pd.read_csv(input_path)
        print(f'-------- {filename} --------')

        # Select relevant columns
        df = df[['DATE (YYYY-MM-DDTHH:MI:SSZ)', 'DATE_QC',
       'LATITUDE (degree_north)', 'LONGITUDE (degree_east)', 'POSITION_QC',
       'PRES (decibar)', 'PRES_QC', 'PSAL (psu)', 'PSAL_QC',
       'TEMP (degree_Celsius)', 'TEMP_QC']]
        
        # Drop NAN rows
        df= df.dropna()

        # Add ID to each sample
        df.insert(0, 'ID', range(1, len(df)+1))

        # Add global QC labels
        df = add_global_qc_label(df)
        
        # Change column names
        df = change_column_names(df)

        # Save the preprocessed data
        df.to_csv(output_path, index=False)

        # Normalize the data
        df = standardize_columns(df, ['Latitude','Longitude','Pressure','Salinity','Temperature'])

        # Normalize the date
        df.insert(1, 'Date', pd.to_datetime(df['Datetime']).dt.date)
        df.drop('Datetime', axis=1, inplace=True)
        df = normalize_date_column(df, date_column='Date', reference_date=pd.Timestamp('2015-01-01'))
        
        # Save the normalized data
        df.to_csv(normalized_path, index=False)


In [ ]:
df